<img src="https://dutchanalytics.com/wp-content/uploads/2020/02/svg_logo_dutch_analytics-1.svg" width="80">

# Quickstart
Deploy a simple float-to-float model in UbiOps using the python client library.


The first step is to download a prepared model.

In [ ]:
!curl -X GET https://storage.googleapis.com/ubiops/example-model-packages/example_model_package.zip -o simple_model_package.zip

Add your API token, provide a project name, model name and model version name. 
Afterward initialize the client library.

In [ ]:
API_TOKEN='Token <YOUR API_KEY>'
PROJECT_NAME='<YOUR PROJECT NAME>'
MODEL_NAME='simple-tutorial'
MODEL_VERSION='v1'

In [ ]:
import ubiops
configuration = ubiops.Configuration()
configuration.api_key['Authorization'] = API_TOKEN
client = ubiops.ApiClient(configuration)
api = ubiops.CoreApi(client)
api.service_status()

## Deploy the model
Create a model.

In [ ]:
model_template = ubiops.ModelCreate(
    name=f'{MODEL_NAME}',
    description='A simple model that multiplies the input float by a random number.',
    input_type='structured',
    output_type='structured',
    input_fields=[
        ubiops.ModelInputFieldCreate(
            name='input',
            data_type='double',
        )
    ],
    output_fields=[
        ubiops.ModelOutputFieldCreate(
            name='output',
            data_type='double'
        )
    ]
)

model = api.models_create(
    project_name=f'{PROJECT_NAME}',
    data=model_template
)
print(model)

Create a model version.

In [ ]:
model_version_template = ubiops.ModelVersionCreate(
    version=f'{MODEL_VERSION}',
    language='python3.7',
    memory_allocation=256,
    maximum_instances=1,
    minimum_instances=0,
    maximum_idle_time=1800 # = 30 minutes
)

model_version = api.model_versions_create(
    project_name=f'{PROJECT_NAME}',
    model_name=f'{MODEL_NAME}',
    data=model_version_template
)
print(model_version)

Upload the prepared model file to the created model version.

In [ ]:
file_upload_result =api.model_versions_file_upload(
    project_name=f'{PROJECT_NAME}',
    model_name=f'{MODEL_NAME}',
    version=f'{MODEL_VERSION}',
    file='simple_model_package.zip'
)

Check if model version is finished building.
This can take a few minutes.

In [ ]:
from time import sleep
status = 'building'
while status != 'available' and 'failed' not in status:    
    model_version_status = api.model_versions_get(       
        project_name=f'{PROJECT_NAME}',        
        model_name=f'{MODEL_NAME}',        
        version=f'{MODEL_VERSION}'    
    )    
    status = model_version_status.status
    sleep(1)
print(status)

## Creating a direct request
Make sure the model is in 'available' state before performing model requests.

In [ ]:
data = {'input': 123}
model_result = api.model_requests_create(
    project_name=f'{PROJECT_NAME}',
    model_name=f'{MODEL_NAME}',
    version=f'{MODEL_VERSION}',
    data=data
)
print(model_result)

## Cleanup
Delete created model.

In [ ]:
api.models_delete(
    project_name=f'{PROJECT_NAME}',
    model_name=f'{MODEL_NAME}'
)